# Generate list of project domains

In [3]:
proj_domains = run_mariadb("""
    select 
        site_global_key as project_key,
        trim(leading "." from reverse(site_domain)) as project_domain
    from
        enwiki.sites
""")

proj_domains.head()

,project_key,project_domain
0,aawiki,aa.wikipedia.org
1,aawiktionary,aa.wiktionary.org
2,aawikibooks,aa.wikibooks.org
3,abwiki,ab.wikipedia.org
4,abwiktionary,ab.wiktionary.org


In [4]:
proj_domains.to_csv("data/raw/project-domains.tsv", sep = "\t", index = False)

# Add the groups of any new Wikipedias

In [7]:
cur_wps = run_mariadb("""
    select 
        site_global_key as project_key,
        trim(leading "." from reverse(site_domain)) as project_domain
    from
        enwiki.sites
    where
        site_group = "wikipedia"
""")

cur_wps.head()

,project_key,project_domain
0,aawiki,aa.wikipedia.org
1,abwiki,ab.wikipedia.org
2,acewiki,ace.wikipedia.org
3,afwiki,af.wikipedia.org
4,akwiki,ak.wikipedia.org


In [15]:
prev_wps = pd.read_csv("data/raw/project_groups.tsv", sep = "\t")
prev_wps.head()

,project_name,project_key,group
0,Afar Wikipedia,aawiki,ssa_wps
1,Abkhazian Wikipedia,abwiki,mena_wps
2,Acehnese Wikipedia,acewiki,asia_wps
3,Adyghe Wikipedia,adywiki,cee_wps
4,Afrikaans Wikipedia,afwiki,ssa_wps


In [16]:
# Find any projects in cur_wps but not in prev_wps
set(cur_wps["project_key"]) - set(prev_wps["project_key"])

{'atjwiki', 'dinwiki', 'dtywiki', 'kbpwiki'}

In [17]:
# Add them manually—congratulations to the pioneer editors in these languages!
new_wps = pd.DataFrame.from_records(
    [["Atikamekw Wikipedia", "atjwiki", "other"], 
     ["Dinka Wikipedia", "dinwiki", "ssa_wps"],
     ["Doteli Wikipedia", "dtywiki", "asia_wps"],
     ["Kabiye Wikipedia", "kbpwiki", "ssa_wps"]],
    columns = ["project_name", "project_key", "group"])

In [20]:
proj_info = prev_wps.append(new_wps)
proj_info = proj_info.sort_values(by = "project_key")

,project_name,project_key,group
0,Afar Wikipedia,aawiki,ssa_wps
1,Abkhazian Wikipedia,abwiki,mena_wps
2,Acehnese Wikipedia,acewiki,asia_wps
3,Adyghe Wikipedia,adywiki,cee_wps
4,Afrikaans Wikipedia,afwiki,ssa_wps


In [21]:
# Make sure it's fixed
set(cur_wps["project_key"]) - set(proj_info["project_key"])

set()

In [23]:
# It's fixed—update the file!
proj_info.to_csv("data/raw/project-groups.tsv", sep = "\t", index = False)